In [1]:
# coding=utf-8
from __future__ import print_function, absolute_import
from gm.api import *

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

## calc
from scipy.stats import linregress

In [2]:
a = pd.DataFrame(
    {'01': [1,2,3,4,5],
     'date': pd.date_range('2025-01-01', periods=5)}
).set_index('date')
a.groupby(pd.Grouper(level=0, freq='MS')).mean()

,01
date,
2025-01-01,3.0


In [3]:
data = pd.read_parquet(r'E:\repo\stocks_dashborads\data\raw\all.parquet')
data

open       high        low      close    volume  \
date       symbol                                                              
2024-01-02 SZSE.002469   5.442895   5.767306   5.442895   5.641147  17807759   
           SHSE.688209  17.762350  17.831810  16.859350  17.018120   7836998   
           SHSE.688208  15.414460  15.466170  15.065450  15.084840   4787896   
           SHSE.600988  13.891490  14.664340  13.841950  14.386910  49931010   
           SHSE.600987   8.089763   8.348044   7.720788   7.776134  15980317   
...                           ...        ...        ...        ...       ...   
2025-08-15 SHSE.688210  36.910000  40.600000  36.710000  39.720000  10384427   
           SHSE.600854   5.410000   5.500000   5.370000   5.390000  10840100   
           SZSE.002393  24.830000  25.480000  24.480000  25.290000  15803173   
           SZSE.002392   6.630000   6.730000   6.620000   6.730000  12765300   
           SHSE.689009  59.830000  61.440000  59.500000  61.160000  14270795   

                                amount  pre_close  
date       symbol                                  
2024-01-02 SZSE.002469  1.11190342e+08   5.460918  
           SHSE.688209  1.35193548e+08  17.722660  
           SHSE.688208  1.12708861e+08  15.349830  
           SHSE.600988  7.24722019e+08  13.881580  
           SHSE.600987  1.39051790e+08   8.080538  
...                                ...        ...  
2025-08-15 SHSE.688210  4.06275427e+08  36.950000  
           SHSE.600854  5.87727610e+07   5.410000  
           SZSE.002393  3.95625736e+08  24.810000  
           SZSE.002392  8.54066240e+07   6.620000  
           SHSE.689009  8.68264900e+08  59.700000  

[1929480 rows x 7 columns]

In [10]:
data[['close']].loc['2025-08-14' : '2025-08-15'].unstack().T.droplevel(0)

date,2025-08-14,2025-08-15
symbol,,
SHSE.603809,15.49,15.96
SZSE.002392,6.62,6.73
SHSE.689009,59.70,61.16
SZSE.002393,24.81,25.29
SHSE.600855,16.58,16.17
...,...,...
SHSE.603357,8.72,8.76
SZSE.300792,28.43,29.30
SZSE.001258,7.11,7.17


In [15]:
6.169629 / 6.178991 - 1

-0.0015151341052285217

In [48]:
data.close.unstack().pct_change(24).stack()

C:\Users\AA\AppData\Local\Temp\ipykernel_19452\2578592114.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data.close.unstack().pct_change(24).stack()


date        symbol     
2024-02-05  SHSE.600000    0.04393937
            SHSE.600004   -0.04423868
            SHSE.600006   -0.14236694
            SHSE.600007   -0.09421063
            SHSE.600008   -0.01470604
                              ...    
2025-08-15  SZSE.301658   -0.03433243
            SZSE.301662    0.07626858
            SZSE.301665   -0.05529075
            SZSE.301678    0.68595371
            SZSE.302132    0.01133246
Length: 1812692, dtype: float64